In [1]:
# pretrained weight 모델을 다운로드 받기 위해서 mmdetection/checkpoints 디렉토리를 만듬. 
#!cd mmdetection; mkdir checkpoints

In [2]:
#!wget -O /content/mmdetection/checkpoints/mask_rcnn_r101_fpn_1x_coco_20200204-1efe0ed5.pth http://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r101_fpn_1x_coco/mask_rcnn_r101_fpn_1x_coco_20200204-1efe0ed5.pth


In [3]:
#!ls -lia /content/mmdetection/checkpoints


In [4]:
import torch
from mmdet.apis import init_detector, inference_detector
import mmcv


print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu111 (GeForce RTX 3070)


In [5]:
# config 파일을 설정하고, 다운로드 받은 pretrained 모델을 checkpoint로 설정. 
config_file = 'configs/mask_rcnn/mask_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = 'checkpoints/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth'
                                

In [6]:
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset

@DATASETS.register_module(force=True)
class NucleusDataset(CocoDataset):
  CLASSES = ['body', 'right_hand', 'left_hand', 'left_foot', 'right_foot', 'right_thigh', 'left_thigh', 'right_calf', 'left_calf', 
     'left_arm', 'right_arm', 'left_forearm',  'right_forearm' , 'head']
  

In [7]:
from mmcv import Config

cfg = Config.fromfile(config_file)


In [8]:
from mmdet.apis import set_random_seed

# dataset에 대한 환경 파라미터 수정. 
cfg.dataset_type = 'NucleusDataset' # 'NucleusDataset'
cfg.data_root = '/home/seonghwan/PyTorch-Simple-MaskRCNN/heroes_data/train_dataset' #'/content/coco_output/'

# train, val, test dataset에 대한 type, data_root, ann_file, img_prefix 환경 파라미터 수정. 
cfg.data.train.type =  'NucleusDataset' #'NucleusDataset'
cfg.data.train.data_root = '/home/seonghwan/PyTorch-Simple-MaskRCNN/heroes_data/train_dataset/' #'/content/coco_output/'
cfg.data.train.ann_file = 'new_ai_2014_train=mask.json' #'train_coco.json'
cfg.data.train.img_prefix = '2014_train=mask'

cfg.data.val.type = 'NucleusDataset' # 'NucleusDataset'
cfg.data.val.data_root = '/home/seonghwan/PyTorch-Simple-MaskRCNN/heroes_data/train_dataset/'#'/content/coco_output/'
cfg.data.val.ann_file ='new_ai_2014_val=mask.json' # 'val_coco.json'
cfg.data.val.img_prefix = '2014_val=mask'

# class의 갯수 수정. 
cfg.model.roi_head.bbox_head.num_classes = 14 # 1
cfg.model.roi_head.mask_head.num_classes = 14 # 1

# pretrained 모델
cfg.load_from = 'checkpoints/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth' #'/content/mmdetection/checkpoints/mask_rcnn_r101_fpn_1x_coco_20200204-1efe0ed5.pth'

# 학습 weight 파일로 로그를 저장하기 위한 디렉토리 설정. 
cfg.work_dir = './tutorial_exps'

# 학습율 변경 환경 파라미터 설정. 
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# CocoDataset의 경우 metric을 bbox로 설정해야 함.(mAP아님. bbox로 설정하면 mAP를 iou threshold를 0.5 ~ 0.95까지 변경하면서 측정)
cfg.evaluation.metric = ['bbox', 'segm']
cfg.evaluation.interval = 12
cfg.checkpoint_config.interval = 12

# epochs 횟수는 36으로 증가 
cfg.runner.max_epochs = 3 

# 두번 config를 로드하면 lr_config의 policy가 사라지는 오류로 인하여 설정. 
cfg.lr_config.policy='step'
# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

In [9]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# train, valid 용 Dataset 생성. 
datasets_train = [build_dataset(cfg.data.train)]
datasets_val = [build_dataset(cfg.data.val)]

loading annotations into memory...
Done (t=11.75s)
creating index...
index created!
loading annotations into memory...
Done (t=0.58s)
creating index...
index created!


In [7]:
datasets_train 

[
 NucleusDataset Train dataset with number of images 26437, and instance counts: 
 +-----------------+-------+-------------------+-------+--------------------+-------+---------------+-------+----------------+-------+
 | category        | count | category          | count | category           | count | category      | count | category       | count |
 +-----------------+-------+-------------------+-------+--------------------+-------+---------------+-------+----------------+-------+
 | 0 [body]        | 50719 | 1 [right_hand]    | 29174 | 2 [left_hand]      | 28062 | 3 [left_foot] | 16352 | 4 [right_foot] | 16334 |
 | 5 [right_thigh] | 28932 | 6 [left_thigh]    | 28546 | 7 [right_calf]     | 21459 | 8 [left_calf] | 21481 | 9 [left_arm]   | 30559 |
 | 10 [right_arm]  | 31574 | 11 [left_forearm] | 29023 | 12 [right_forearm] | 30216 | 13 [head]     | 38272 | -1 background  | 0     |
 +-----------------+-------+-------------------+-------+--------------------+-------+---------------+------

In [11]:
print(datasets_train[0])
print(datasets_val[0])


NucleusDataset Train dataset with number of images 26437, and instance counts: 
+-----------------+-------+-------------------+-------+--------------------+-------+---------------+-------+----------------+-------+
| category        | count | category          | count | category           | count | category      | count | category       | count |
+-----------------+-------+-------------------+-------+--------------------+-------+---------------+-------+----------------+-------+
| 0 [body]        | 50719 | 1 [right_hand]    | 29174 | 2 [left_hand]      | 28062 | 3 [left_foot] | 16352 | 4 [right_foot] | 16334 |
| 5 [right_thigh] | 28932 | 6 [left_thigh]    | 28546 | 7 [right_calf]     | 21459 | 8 [left_calf] | 21481 | 9 [left_arm]   | 30559 |
| 10 [right_arm]  | 31574 | 11 [left_forearm] | 29023 | 12 [right_forearm] | 30216 | 13 [head]     | 38272 | -1 background  | 0     |
+-----------------+-------+-------------------+-------+--------------------+-------+---------------+-------+-------

In [10]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets_train[0].CLASSES
print(model.CLASSES)

['body', 'right_hand', 'left_hand', 'left_foot', 'right_foot', 'right_thigh', 'left_thigh', 'right_calf', 'left_calf', 'left_arm', 'right_arm', 'left_forearm', 'right_forearm', 'head']


In [10]:


import mmcv
import os.path as osp
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# epochs는 config의 runner 파라미터로 지정됨. 기본 12회 
train_detector(model, datasets_train, cfg, distributed=False, validate=True)

loading annotations into memory...


2021-11-22 18:30:40,301 - mmdet - INFO - load checkpoint from local path: checkpoints/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth


Done (t=0.35s)
creating index...
index created!


2021-11-22 18:30:41,503 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([15, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([15]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([56, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([56]).
size mismatch for roi_head.mask_head.conv_logits.weight: copying a param with shape torch.Size([80, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([14, 256, 1, 1]).
size mismatch for roi_head.mask_head.conv_logits

In [11]:
from mmdet.apis import inference_detector, show_result_pyplot

checkpoint_file = './tutorial_exps/epoch_50.pth'

# checkpoint 저장된 model 파일을 이용하여 모델을 생성, 이때 Config는 위에서 update된 config 사용. 
model_ckpt = init_detector(cfg, checkpoint_file, device='cuda:0')

load checkpoint from local path: ./tutorial_exps/epoch_50.pth


In [12]:
import cv2
# val_img_name = val_df['image_name'].iloc[0]
# print(val_img_name)

# BGR Image 사용 

img_arr = cv2.imread('/home/seonghwan/PyTorch-Simple-MaskRCNN/heroes_data/train_dataset/2014_val=mask/COCO_val2014_000000001000.png')
model_ckpt.cfg = cfg

results = inference_detector(model_ckpt, img_arr)
show_result_pyplot(model_ckpt, img_arr, results, score_thr=0.1)

/home/seonghwan/mmdetection/mmdet/datasets/utils.py:65: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
/home/seonghwan/anaconda3/envs/mask_rcnn_ver2/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/seonghwan/mmdetection/mmdet/core/visualization/image.py:183: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [13]:
import numpy as np
labels_to_names_seq = [{1:'body'},
                    {2:'right_hand'},
                    {3:'left_hand'},
                    {4:'left_foot'},
                    {5:'right_foot'},
                    {6:'right_thigh'},
                    {7:'left_thigh'},
                    {8:'right_calf'},
                    {9:'left_calf'},
                    {10:'left_arm'},
                    {11:'right_arm'},
                    {12:'left_forearm'},
                    {13:'right_forearm'},
                    {14:'head'}]
  
COLORS = list(
    [[0, 255, 0],
     [0, 0, 255],
     [255, 0, 0],
     [0, 255, 255],
     [255, 255, 0],
     [255, 0, 255],
     [80, 70, 180],
     [250, 80, 190],
     [245, 145, 50],
     [70, 150, 250],
     [70, 0 ,0],
     [141, 200 ,250],
     [250, 70 ,255],
     [255, 70 , 70],
     [111, 1 ,25],
     [111, 70 ,255],
     [210, 52 ,177],
])

In [14]:
# model과 원본 이미지 array, filtering할 기준 class confidence score를 인자로 가지는 inference 시각화용 함수 생성. 
# 이미 inference 시 mask boolean값이 들어오므로 mask_threshold 값을 필요하지 않음. 
def get_detected_img(model, img_array,  score_threshold=0.3, draw_box=True, is_print=True):
  # 인자로 들어온 image_array를 복사. 
  draw_img = img_array.copy()
  bbox_color=(0, 255, 0)
  text_color=(0, 0, 255)

  # model과 image array를 입력 인자로 inference detection 수행하고 결과를 results로 받음.  
  results = inference_detector(model, img_array)
  bbox_results = results[0]
  seg_results = results[1]

  # results 리스트를 loop를 돌면서 개별 2차원 array들을 추출하고 이를 기반으로 이미지 시각화 
  # results 리스트의 위치 index가 매핑된 Class id. 여기서는 result_ind가 class id
  # 개별 2차원 array에 오브젝트별 좌표와 class confidence score 값을 가짐. 
  for result_ind, bbox_result in enumerate(bbox_results):
    # 개별 2차원 array의 row size가 0 이면 해당 Class id로 값이 없으므로 다음 loop로 진행. 
    if len(bbox_result) == 0:
      continue
    
    mask_array_list = seg_results[result_ind]
    
    # 해당 클래스 별로 Detect된 여러개의 오브젝트 정보가 2차원 array에 담겨 있으며, 이 2차원 array를 row수만큼 iteration해서 개별 오브젝트의 좌표값 추출. 
    for i in range(len(bbox_result)):
      # 좌상단, 우하단 좌표 추출. 
      if bbox_result[i, 4] > score_threshold:
        left = int(bbox_result[i, 0])
        top = int(bbox_result[i, 1])
        right = int(bbox_result[i, 2])
        bottom = int(bbox_result[i, 3])
        caption = "{}: {:.4f}".format(labels_to_names_seq[result_ind], bbox_result[i, 4])
        if draw_box:
          cv2.rectangle(draw_img, (left, top), (right, bottom), color=bbox_color, thickness=2)
          cv2.putText(draw_img, caption, (int(left), int(top - 7)), cv2.FONT_HERSHEY_SIMPLEX, 0.37, text_color, 1)
        # masking 시각화 적용. class_mask_array는 image 크기 shape의  True/False값을 가지는 2차원 array
        class_mask_array = mask_array_list[i]
        # 원본 image array에서 mask가 True인 영역만 별도 추출. 
        masked_roi = draw_img[class_mask_array]
        
        #color를 임의 지정
        # color_index = np.random.randint(0, len(COLORS)-1)
        # color를 class별로 지정
        color_index = result_ind % len(COLORS)
        color = COLORS[color_index]

        # apply_mask()함수를 적용시 수행 시간이 상대적으로 오래 걸림. 
        #draw_img = apply_mask(draw_img, class_mask_array, color, alpha=0.4)
        # 원본 이미지의 masking 될 영역에 mask를 특정 투명 컬러로 적용
        draw_img[class_mask_array] = ([0.3*color[0], 0.3*color[1], 0.3*color[2]] + 0.6 * masked_roi).astype(np.uint8)
        
        if is_print:
          print(caption)
  
  return draw_img

In [15]:
from PIL import Image
cv2.imwrite('image/baps_image.png',img_arr)

True

In [17]:
cv2.imwrite('image/segmentation.png',detected_img)

True

In [15]:
# Line extraction

In [16]:
# inference mask detected 된 이미지 보기
import matplotlib.pyplot as plt
from glob import glob 
from tqdm import tqdm
from PIL import Image
image_list = []
path = glob('/media/seonghwan/1.8T_Backup_HDD/Label/2017/6/*.png')

for i in tqdm(path):
    
    
    img = cv2.imread(i)
    image_list.append(img)


100%|██████████| 614/614 [00:31<00:00, 19.20it/s]


In [34]:
img_arr = image_list[20]
detected_img = get_detected_img(model_ckpt, img_arr,  score_threshold=0.48, draw_box=True, is_print=True)
detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2RGB)
cv2.imwrite('result_test/seg1.png',detected_img)

{1: 'body'}: 0.9965
{3: 'left_hand'}: 0.7329
{3: 'left_hand'}: 0.7213
{4: 'left_foot'}: 0.5125
{4: 'left_foot'}: 0.4902
{5: 'right_foot'}: 0.4921
{6: 'right_thigh'}: 0.5178
{6: 'right_thigh'}: 0.4858
{7: 'left_thigh'}: 0.5298
{7: 'left_thigh'}: 0.5246
{8: 'right_calf'}: 0.5031
{8: 'right_calf'}: 0.4953
{9: 'left_calf'}: 0.5227
{9: 'left_calf'}: 0.5185
{10: 'left_arm'}: 0.6430
{10: 'left_arm'}: 0.5905
{12: 'left_forearm'}: 0.5924
{12: 'left_forearm'}: 0.5474
{13: 'right_forearm'}: 0.4855
{13: 'right_forearm'}: 0.4806
{14: 'head'}: 0.9666


True